In [1]:
%reset -f


In [2]:
import ee
ee.Authenticate()


In [3]:
import ee

ee.Initialize(project='ee-andrewfullhart')

nex_im = ee.ImageCollection('NASA/NEX-DCP30').first()
scale = nex_im.projection().nominalScale().getInfo()

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

orig_proj = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').filterBounds(area_shp).first().projection()
elev_im = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM').mosaic().setDefaultProjection(orig_proj)
proj = elev_im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = elev_im.projection()

e_elev_im = elev_im.setDefaultProjection('EPSG:4326', transform)
e_elev_im = e_elev_im.reproject(crs=proj.crs(), crsTransform=transform_new)
e_elev_im = e_elev_im.clip(area_shp)

slope_im = ee.Terrain.slope(elev_im)
e_slope_im = slope_im.setDefaultProjection('EPSG:4326', transform)
e_slope_im = e_slope_im.reproject(crs=proj.crs(), crsTransform=transform_new)
e_slope_im = e_slope_im.clip(area_shp)


task = ee.batch.Export.image.toDrive(
  image=e_elev_im,
  description='DEM_Elev',
  folder='GEE_Downloads',
  region=area_shp,
  scale=scale,
  crs='EPSG:4326',
  maxPixels=1e13)

task.start()

task = ee.batch.Export.image.toDrive(
  image=e_slope_im,
  description='DEM_Slope',
  folder='GEE_Downloads',
  region=area_shp,
  scale=scale,
  crs='EPSG:4326',
  maxPixels=1e13)

task.start()


/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JAXA/ALOS/AW3D30/V3_2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_AW3D30_V3_2

  warnings.warn(warning, category=DeprecationWarning)
